In [1]:
#Libraries
import pymongo
from geopy.geocoders import Nominatim
import folium
import pandas as pd

In [19]:
#functions

#connection with MongoDB
def con_mongo(url,database,collection):
  
  myclient = pymongo.MongoClient(url)
  
  dblist = myclient.list_database_names()
  
  if database in dblist:
   print(f'The database {database} exists')
   mydb = myclient[database]
   collectionlist = mydb.list_collection_names()
   
   if collection in collectionlist:
    print(f'Collection {collection} exists')
    mycol = mydb[collection]
   else: 
     mycol = mydb[collection]
     print(f'Collection {collection} created')
  
  else:
    mydb = myclient[database]
    mycol = mydb[collection]
    print(f'database {database} created')
    print(f'Collection {collection} created')
  #retunr connection  
  return mycol

#read CSV dataset with school information
def read_csv(path):    
    
    dict_mongo = {}
    lista = []
    #choosing specific columns
    columns = ['UNIDADE_ ESCOLAR','LATITUDE','LONGITUDE']
    
    df = pd.read_csv(path, sep=';', encoding='ANSI', usecols=columns)

    #removing lines NAN
    df=df.dropna(axis=0)
    
    df['UNIDADE_ ESCOLAR'] = df['UNIDADE_ ESCOLAR'].str.title()
    df['LATITUDE'] = df['LATITUDE'].str.replace(',','.').astype(float) 
    df['LONGITUDE'] = df['LONGITUDE'].str.replace(',','.').astype(float)

    #remove invalid coordinates
    invalid_coordinates = df[(df['LONGITUDE'] < -180) | (df['LONGITUDE'] > 180) | (df['LATITUDE'] < -90) | (df['LATITUDE'] > 90)].index

    df.drop(invalid_coordinates , inplace=True)

    #droping index
    df.reset_index(drop=True,inplace=True)

    #creating a dict for each row with information in geospatial format
    for i, d in df.iterrows():
        dict_mongo = {
                    'UNIDADE_ ESCOLAR':d['UNIDADE_ ESCOLAR'],
                    'LATITUDE':d['LATITUDE'],
                    'LONGITUDE':d['LONGITUDE'],
                    'loc':{
                        'type': "Point", 'coordinates': [d['LATITUDE'], d['LONGITUDE']]
                    }
        }
        lista.append(dict_mongo)
    return lista

#searching nearby school
def search_school_nearby(coordinates,maxdistance, connection):
  #convert maxdistance value to KM
  maxdistance *= 1000
  
  #Query Mongodb
  myquery =  {'loc': {'$geoNear' : {'$geometry': {'type' : '"Point"', 'coordinates': coordinates}, '$maxDistance': maxdistance}}}  

  mydoc = connection.find(myquery)
  result = []

  for x in mydoc:
    dict_mongo = {
                  'UNIDADE_ ESCOLAR':x['UNIDADE_ ESCOLAR'],
                  'LATITUDE':x['LATITUDE'],
                  'LONGITUDE':x['LONGITUDE']
                }
    result.append(dict_mongo)
  return result

#plotting Map with result 
def plot_map (result,coordinates):
  map=folium.Map(location=coordinates,zoom_start=13)
  map.add_child(folium.Marker(location=coordinates,popup='You Are Here',icon=folium.Icon(color='green')))
  
  if len(result) > 0:
    for x in result:
      map.add_child(folium.Marker(location=[x['LATITUDE'], x['LONGITUDE']],popup=x['UNIDADE_ ESCOLAR'],icon=folium.Icon(color='red')))
  return map


In [3]:
#conneciton mongo
url_mongo = "mongodb://localhost:27017/"
database = "coordinate"
collection = "school"

#Inform your coordinates
coordinates = [-3.724725, -38.545837]#[-3.7379357, -3.7378275]

file = '4.4.3-unidades-educacionais-no-municipio_31.12.2023.csv'

dataset = read_csv(file)

#create connection with MongoDB
conn = con_mongo(url_mongo,database,collection)

#Insert dataset 
conn.insert_many(dataset)

#Create spatial index
conn.create_index({'loc': "2dsphere"})

database coordinate created
Collection school created


'loc_2dsphere'

In [24]:
maxdistance = 0.5 #distance in KM
#search schools
result = search_school_nearby(coordinates,maxdistance, conn)

print(f'There are {len(result)} schools nearby you !')

# #ploting map
mapa = plot_map (result,coordinates)

# #result
mapa


There are 3 schools nearby you !
